Datatset used - https://www.kaggle.com/datasets/paultimothymooney/poetry

In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [2]:
path = "./lyrics/"

def read_all(path):
    txt = []
    for _, _, files in os.walk(path):
        for file in files:
            with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                in_lyric = False
                for line in f:
                    line = line.strip()
                    if line == '----------':
                        if in_lyric:
                            in_lyric = False
                            txt.append("")
                    elif line:
                        if line.isupper() and line.isalpha():
                            in_lyric = True 
                        if in_lyric:
                            txt.append(line) 
    return txt
lyrics = read_all(path)
print("number of lines = ", len(lyrics))


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 44: character maps to <undefined>

In [ ]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_stories = clean_txt(lyrics)
print("number of words = ", len(cleaned_stories))

In [ ]:
print(cleaned_stories[:5])

In [ ]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

In [ ]:
markov_model = make_markov_model(cleaned_stories)

In [ ]:
print("number of states = ", len(markov_model.keys()))

In [ ]:
print("All possible transitions from 'your love' state: \n")
print(markov_model['your love'])

In [ ]:
def generate_story(markov_model, limit=100, start='your love'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))
        
        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story

In [ ]:
for i in range(1,9):
    print("-> ", generate_story(markov_model, start="can you", limit=8))

In [ ]:
for i in range(1,9):
    print("-> ", generate_story(markov_model, start="baby i", limit=8))